# function call

## 简介

function_call，顾名思义，通过给大模型提供function的说明描述，以及对应的入参出参schema，让大模型输出function调用策略，结合多轮对话，以最终实现一个复杂的任务。
以下将以天气获取为例子，通过qianfan SDK提供的ERNIE-Bot大模型以实现通过大模型得到对应城市的天气情况。

## 准备

本文使用了Langchain中的的agent以及tool以实现agent的中间步骤传递转移逻辑，所以我们需要安装qianfan SDK和langchain， 本文环境使用的版本如下所示：


In [ ]:
!pip install qianfan==0.1.3
!pip install langchain==0.0.335

In [1]:
# 初始化LLM
import os
from langchain.chat_models import QianfanChatEndpoint

os.environ["QIANFAN_AK"] = "your_ak"
os.environ["QIANFAN_SK"] = "your_sk"
llm = QianfanChatEndpoint(model="ERNIE-Bot") # 仅EB支持


### 构造prompt

In [2]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at get today's temperature of location."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

### 定义functions

这一步需要定义好使用的functions的参数和描述，这会影响到LLm对function_call的回答

In [3]:
FUNCTION_SCHEMA_GET_WEATHER = {
    "name": "get_current_weather",
    "description": "获得指定地点的天气",
    "parameters": {
        "type": "object",
        "properties": {
        "location": {
            "type": "string",
            "description": "省，市名，例如：河北省，石家庄"
        },
        "unit": {
            "type": "string",
            "enum": ["摄氏度", "华氏度"]
        }
        },
        "required": ["location"]
    }
}


from langchain.agents import tool

@tool
def get_current_weather(location: str) -> str:
    """Returns current temperature of location."""
    return "25"

tools = [get_current_weather]

In [4]:
llm_with_tools = llm.bind(
    functions=[
        FUNCTION_SCHEMA_GET_WEATHER,
    ]
)

In [7]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions

p = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
} 
agent = p | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()


In [8]:
# main loop for functions steps
# 由于erniebot本身system对外有一定限制，所以会报warning
from langchain.schema.agent import AgentFinish
intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "上海市今天天气如何？",
        "intermediate_steps": intermediate_steps
    })
    
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_current_weather": get_current_weather,
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

[WARNING] [09-27 17:22:00] logging.py:92 [t:140650105567040]: This key `system` does not seem to be a parameter that the model `ERNIE-Bot` will accept
[WARNING] [09-27 17:22:02] logging.py:92 [t:140650105567040]: This key `system` does not seem to be a parameter that the model `ERNIE-Bot` will accept


get_current_weather {'location': '上海市', 'unit': '摄氏度'}
上海市今天的天气是25摄氏度。
